# Libraries

In [39]:
import pandas as pd
import numpy as np

# Import

In [40]:
fueleconomy = pd.read_pickle("../FuelEconomy/cleanedfueleconomy.pkl")
registrations = pd.read_pickle("../VehicleRegistrations/cleanedregistrations.pkl")
consumption = pd.read_pickle("../ConsumptionData/cleanedconsumption.pkl")

In [41]:
consumption.info()
print("\n")
fueleconomy.info()
print("\n")
registrations.info()

<class 'pandas.core.frame.DataFrame'>
Index: 324 entries, 600 to 923
Data columns (total 3 columns):
 #   Column                                     Non-Null Count  Dtype         
---  ------                                     --------------  -----         
 0   Date                                       324 non-null    datetime64[ns]
 1   Gasoline_Consumption(Millions of Gallons)  324 non-null    float64       
 2   Z_Consumption                              324 non-null    float64       
dtypes: datetime64[ns](1), float64(2)
memory usage: 10.1 KB


<class 'pandas.core.frame.DataFrame'>
Index: 27 entries, 7 to 33
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Year           27 non-null     datetime64[ns]
 1   Car            27 non-null     float64       
 2   Car SUV        27 non-null     float64       
 3   Pickup truck   27 non-null     float64       
 4   Van            27 non-null     flo

In [42]:
''' Drop Unneeded Columns and Rename Needed Columns if Needed'''
fueleconomy = fueleconomy.iloc[:, :6]
registrations = registrations.iloc[:, :4]
consumption = consumption.iloc[:, :2]
consumption.columns = ["Date", "Gasoline_Consumption(Millions of Gallons)"]

In [43]:
''' Converting Consumption data to Years '''

# convert consumption tp yearly and sum up values
consumption = consumption.rename(columns = {'Date' : 'Year'})
consumption = (consumption.groupby('Year', as_index=False)['Gasoline_Consumption(Millions of Gallons)']).sum()

In [ ]:
fueleconomy = (fueleconomy)

In [44]:
consumption.info()
print("\n")
fueleconomy.info()
print("\n")
registrations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 324 entries, 0 to 323
Data columns (total 2 columns):
 #   Column                                     Non-Null Count  Dtype         
---  ------                                     --------------  -----         
 0   Year                                       324 non-null    datetime64[ns]
 1   Gasoline_Consumption(Millions of Gallons)  324 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 5.2 KB


<class 'pandas.core.frame.DataFrame'>
Index: 27 entries, 7 to 33
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Year          27 non-null     datetime64[ns]
 1   Car           27 non-null     float64       
 2   Car SUV       27 non-null     float64       
 3   Pickup truck  27 non-null     float64       
 4   Van           27 non-null     float64       
 5   Truck SUV     27 non-null     float64       
dtypes: datetime64[ns]

# Merge

# in progress 

In [46]:
''' Merge consumption and prices data on Date '''

# Normalize and align Date columns just in case
consumption["Year"] = pd.to_datetime(consumption["Year"]).dt.tz_localize(None).dt.normalize()
fueleconomy["Year"] = pd.to_datetime(fueleconomy["Year"]).dt.tz_localize(None).dt.normalize()
registrations["Year"] = pd.to_datetime(registrations["Year"]).dt.tz_localize(None).dt.normalize()

# Merge consumption and fueleconomy data
merged_df = pd.merge(consumption, fueleconomy, on="Year", how="inner")
merged_df = pd.merge(merged_df, registrations, on="Year", how="inner")

# ------^  edited ^------#




'''
# Keep only relevant columns
merged_df = merged_df[[
    "Date",
    "GasolinePrices",
    "DieselPrices",
    "Gasoline_Consumption(Millions of Gallons)"
]]

# Rename for clarity
merged_df.rename(columns={
    "Dollars_Per_Gallon": "Nominal_Price_USD",
    "price_adjusted": "Inflation_Adjusted_Price_USD"
}, inplace=True)

# Round values for cleaner display
merged_df["Nominal_Price_USD"] = merged_df["Nominal_Price_USD"].round(3)
merged_df["Inflation_Adjusted_Price_USD"] = merged_df["Inflation_Adjusted_Price_USD"].round(3)

# Add time breakdowns
merged_df["Year"] = merged_df["Date"].dt.year
merged_df["Month"] = merged_df["Date"].dt.month_name()

'''

# Verify output
print(merged_df.tail(10))

         Year  Gasoline_Consumption(Millions of Gallons)    Car  Car SUV  \
17 2012-01-01                                  10663.926  27.60    23.29   
18 2013-01-01                                  10847.088  28.36    24.34   
19 2014-01-01                                  10772.034  28.39    24.44   
20 2015-01-01                                  11248.104  29.01    25.14   
21 2016-01-01                                  11266.584  29.21    26.21   
22 2017-01-01                                  11075.736  30.18    26.14   
23 2018-01-01                                  11441.430  30.76    27.33   
24 2019-01-01                                  11429.460  30.88    27.49   
25 2020-01-01                                  11358.060  31.73    28.38   
26 2021-01-01                                  10055.766  31.73    30.50   

    Pickup truck    Van  Truck SUV  Automobile_Registrations  \
17         17.21  21.27      20.01              1.112899e+08   
18         17.47  21.06      20.83 